In [8]:
import sys
import os
top_path = os.path.abspath('../')
if top_path not in sys.path:
    sys.path.insert(0, top_path)

# Third-party
from astropy.io import fits
import astropy.units as u
import matplotlib as mpl
import numpy as np

from gwb.likelihood import get_tangent_basis

In [17]:
colnames = ['ra', 'dec', 'parallax', 'pmra', 'pmdec', 'Cinv',
            'true_dist', 'true_vx', 'true_vy', 'true_vz']
dtypes = [('f8',), ('f8',), ('f8',), ('f8',), ('f8',), ('f8', (4,4)),
          ('f8',), ('f8',), ('f8',), ('f8',)]
dtype = [(name,)+dt for name,dt in zip(colnames, dtypes)]

In [2]:
def make_pair_same_v(d1, sep_pc):
    """
    Given a distance to star 1, and a physical separation, generate mock 
    data for a pair of stars with the same 3-space velocity.
    """
    
    # 3-space, cartesian velocity vector
    true_v = np.random.normal(0, 30., size=3)
    
    # random position
    ra1 = np.random.uniform(0, 2*np.pi)
    dec1 = np.pi/2. - np.arccos(2*np.random.uniform() - 1.)
    v1 = get_tangent_basis(ra1, dec1).dot(true_v)

    ra2 = ra1 + np.random.choice([1.,-1.])*sep_pc/np.sqrt(2.)/d1
    dec_choice = np.random.choice([1.,-1.])
    dec2 = dec1 + dec_choice*sep_pc/np.sqrt(2.)/d1
    if dec2 > np.pi/2 or dec2 < -np.pi/2.:
        dec2 += -2*dec_choice*sep_pc/np.sqrt(2.)/d1

    d2 = d1 + np.random.choice([1.,-1.])*sep_pc/np.sqrt(2.)
    v2 = get_tangent_basis(ra2, dec2).dot(true_v)

    # picked at random from TGAS data
    Cov1 = np.array([[0.066019, 0.0581179, -0.06287875],
                     [0.0581179, 2.76016904, -1.31836247],
                     [-0.06287875, -1.31836247, 0.83536227]])
    Cov2 = np.array([[0.07143981, 0.31518988, -0.11283286],
                     [0.31518988, 6.78049538, -2.21299533],
                     [-0.11283286, -2.21299533, 0.9342514]])

    plx1 = (d1*u.pc).to(u.mas,u.parallax()).value
    plx2 = (d2*u.pc).to(u.mas,u.parallax()).value

    pm1 = (v1[:2] / d1 * u.km/u.s/u.pc).to(u.mas/u.yr, u.dimensionless_angles()).value
    pm2 = (v2[:2] / d2 * u.km/u.s/u.pc).to(u.mas/u.yr, u.dimensionless_angles()).value

    true_x1 = np.concatenate(([plx1],pm1))
    true_x2 = np.concatenate(([plx2],pm2))

    x1 = np.zeros(4) # ignore vr
    x1[:3] = np.random.multivariate_normal(true_x1, Cov1)

    x2 = np.zeros(4) # ignore vr
    x2[:3] = np.random.multivariate_normal(true_x2, Cov2)

    Cinv1 = np.zeros((4,4))
    Cinv1[:3,:3] = np.linalg.inv(Cov1)

    Cinv2 = np.zeros((4,4))
    Cinv2[:3,:3] = np.linalg.inv(Cov2)

    return (ra1,dec1,x1[0],x1[1],x1[2],Cinv1), (ra2,dec2,x2[0],x2[1],x2[2],Cinv2), true_x1, true_x2, true_v

In [16]:
filename1 = '../data/mock-same-vel-pairs-1.fits'
filename2 = '../data/mock-same-vel-pairs-2.fits'

rows1 = []
rows2 = []
for d1 in [51.41, 154.68, 223.893]:
    for sep_pc in [1E-1, 1., 3.]:
        data1, data2, true_x1, true_x2, true_v = make_pair_same_v(d1, sep_pc)
        
        row1 = data1 + (1000/true_x1[0],) + tuple(true_v)
        rows1.append(row1)
        
        row2 = data2 + (1000/true_x2[0],) + tuple(true_v)
        rows2.append(row2)

hdu1 = fits.BinTableHDU(np.array(rows1, dtype=dtype))
hdu1.writeto(filename1)

hdu2 = fits.BinTableHDU(np.array(rows2, dtype=dtype))
hdu2.writeto(filename2)

---

## Pairs with different true velocities

In [18]:
def make_pair_diff_v(d1, sep_pc):
    """
    Given a distance to star 1, and a physical separation, generate mock 
    data for a pair of stars with the same 3-space velocity.
    """
    
    true_v1 = np.random.normal(0, 30., size=3)
    true_v2 = np.random.normal(0, 30., size=3)

    ra1 = np.random.uniform(0, 2*np.pi)
    dec1 = np.pi/2. - np.arccos(2*np.random.uniform() - 1.)
    v1 = get_tangent_basis(ra1, dec1).dot(true_v1)

    ra2 = ra1 + np.random.choice([1.,-1.])*sep_pc/np.sqrt(2.)/d1
    dec_choice = np.random.choice([1.,-1.])
    dec2 = dec1 + dec_choice*sep_pc/np.sqrt(2.)/d1
    if dec2 > np.pi/2 or dec2 < -np.pi/2.:
        dec2 += -2*dec_choice*sep_pc/np.sqrt(2.)/d1

    d2 = d1 + np.random.choice([1.,-1.])*sep_pc/np.sqrt(2.)
    v2 = get_tangent_basis(ra2, dec2).dot(true_v2)

    # picked at random from TGAS data
    Cov1 = np.array([[0.066019, 0.0581179, -0.06287875],
                     [0.0581179, 2.76016904, -1.31836247],
                     [-0.06287875, -1.31836247, 0.83536227]])
    Cov2 = np.array([[0.07143981, 0.31518988, -0.11283286],
                     [0.31518988, 6.78049538, -2.21299533],
                     [-0.11283286, -2.21299533, 0.9342514]])

    plx1 = (d1*u.pc).to(u.mas,u.parallax()).value
    plx2 = (d2*u.pc).to(u.mas,u.parallax()).value

    pm1 = (v1[:2] / d1 * u.km/u.s/u.pc).to(u.mas/u.yr, u.dimensionless_angles()).value
    pm2 = (v2[:2] / d2 * u.km/u.s/u.pc).to(u.mas/u.yr, u.dimensionless_angles()).value

    true_x1 = np.concatenate(([plx1],pm1))
    true_x2 = np.concatenate(([plx2],pm2))

    x1 = np.zeros(4) # ignore vr
    x1[:3] = np.random.multivariate_normal(true_x1, Cov1)

    x2 = np.zeros(4) # ignore vr
    x2[:3] = np.random.multivariate_normal(true_x2, Cov2)

    Cinv1 = np.zeros((4,4))
    Cinv1[:3,:3] = np.linalg.inv(Cov1)

    Cinv2 = np.zeros((4,4))
    Cinv2[:3,:3] = np.linalg.inv(Cov2)

    return (ra1,dec1,x1[0],x1[1],x1[2],Cinv1), (ra2,dec2,x2[0],x2[1],x2[2],Cinv2), true_x1, true_x2, true_v1, true_v2

In [19]:
filename1 = '../data/mock-diff-vel-pairs-1.fits'
filename2 = '../data/mock-diff-vel-pairs-2.fits'

rows1 = []
rows2 = []
for d1 in [51.41, 154.68, 223.893]:
    for sep_pc in [1E-1, 1., 3.]:
        data1, data2, true_x1, true_x2, true_v1, true_v2 = make_pair_diff_v(d1, sep_pc)
        
        row1 = data1 + (1000/true_x1[0],) + tuple(true_v1)
        rows1.append(row1)
        
        row2 = data2 + (1000/true_x2[0],) + tuple(true_v2)
        rows2.append(row2)

hdu1 = fits.BinTableHDU(np.array(rows1, dtype=dtype))
hdu1.writeto(filename1)

hdu2 = fits.BinTableHDU(np.array(rows2, dtype=dtype))
hdu2.writeto(filename2)